In [5]:
# 1. 데이터 준비 및 확인

import pandas as pd
import urllib.request
%matplotlib inline
import matplotlib.pyplot as plt
import re


from konlpy.tag import Okt
from konlpy.tag import Mecab
# pip install konlyp 를 통해 설치

from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from collections import Counter


train_data = pd.read_table('~/sentiment_classification/ratings_train.txt')
test_data = pd.read_table('~/sentiment_classification/ratings_test.txt')

train_data.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [6]:
# 2. 데이터로더 구성

# imdb.data.loader() 메소드를 호출하면 숫자 인덱스로 변환된 텍스트와
#  word_to_index 딕셔너리까지 친절하게 제공한다. 
#  하지만 nsmc 데이터셋은 전혀 가공되지 않은 텍스트 파일로 이루어져있어, 
#  imdb_loader() 와 동일하게 동작하는 data_loader 를 만들어야 한다. 
#  data_loader 안에서는 다음을 수행해야 한다.

#    1. 데이터의 중복제거
#    2. 결측치 제거
#    3. 한국어 토크나이저로 토큰화
#    4. 불용어(Stopwords) 제거
#    5. 사전 `word_to_index` 구성
#    6. 텍스트 스트링을 사전 인텍스 스티링으로 변환
#    7. X_train, y_train, X_test, y_tesr, word_to_index 리턴



# 데이터 중복 확인

# id는 150000개 인데, document 는 146182개면, document 에서 약 4000개의 자료가 중복
print(train_data.nunique())

print(test_data.nunique())

id          150000
document    146182
label            2
dtype: int64
id          50000
document    49157
label           2
dtype: int64


In [7]:
# 토크나이저 생성

tokenizer = Mecab()

# 불용어
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

def load_data(train_data, test_data, num_words=10000):
    # 위에서 확인한 document 컬럼 값의 중복을 제거하고 저장
    train_data.drop_duplicates(subset=['document'], inplace=True)
    # 결측치 제거
    train_data = train_data.dropna(how = 'any') 
    test_data.drop_duplicates(subset=['document'], inplace=True)
    test_data = test_data.dropna(how = 'any') 

    X_train = []
    for sentence in train_data['document']:
        temp_X = tokenizer.morphs(sentence) # 토큰화
        temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
        X_train.append(temp_X)

    X_test = []
    for sentence in test_data['document']:
        temp_X = tokenizer.morphs(sentence) # 토큰화
        temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
        X_test.append(temp_X)

    words = np.concatenate(X_train).tolist()
    counter = Counter(words)
    counter = counter.most_common(10000-4)
    vocab = ['<PAD>', '<BOS>', '<UNK>', '<UNUSED>'] + [key for key, _ in counter]
    word_to_index = {word:index for index, word in enumerate(vocab)}

    def wordlist_to_indexlist(wordlist):
        return [word_to_index[word] if word in word_to_index else word_to_index['<UNK>'] for word in wordlist]

    X_train = list(map(wordlist_to_indexlist, X_train))
    X_test = list(map(wordlist_to_indexlist, X_test))

    return X_train, np.array(list(train_data['label'])), X_test, np.array(list(test_data['label'])), word_to_index

X_train, y_train, X_test, y_test, word_to_index = load_data(train_data, test_data)

# 훈련 데이터 셋, 테스트 데이터 셋 확인
print("훈련 샘플 개수 :{}, 테스트 개수 : {}".format(len(X_train),len(X_test)))

훈련 샘플 개수 :146182, 테스트 개수 : 49157


In [ ]:
# 